In [28]:
import numpy as np
import math
import copy

### Part 1: Evaluating diffrerent objectives

In [27]:
data = np.array([[1, 2, 1, 2, 10, 10.3, 10.5, 10.7],
                 [1, 1, 2, 2,  2,  2,  2, 2]])
labels = np.array([[-1, -1, 1, 1, 1, 1, 1, 1]])
blue_th = np.array([[0, 1]]).T
blue_th0 = -1.5
red_th = np.array([[1, 0]]).T
red_th0 = -2.5

def margin(data, label, th: np.ndarray, th0):
    return label * (np.dot(data, th) + th0) / np.linalg.norm(th)

red_m = np.array([margin(data[:,i], labels[:,i], red_th, red_th0) for i in range(len(data[0]))])
blue_m = np.array([margin(data[:,i], labels[:,i], blue_th, blue_th0) for i in range(len(data[0]))])
print([red_m.sum(), red_m.min(), red_m.max()])
print([blue_m.sum(), blue_m.min(), blue_m.max()])

[31.5, -1.5, 8.2]
[4.0, 0.5, 0.5]


### Part 2 - 5: SVM (support vector machine) 
In some models, we are not simply seperating data by a slim plane. Instead, we should also maximize the margin bewteen two classes.

The idea is that if we have some $\gamma_{\mathrm{ref}}$ that we would like to make margin as large as $\gamma_{\mathrm{ref}}$. Take a look at the generic objective as 
$$ J(\theta, \theta_0) = \frac 1 n \sum L(x^{(i)}, y^{(i)}, \theta, \theta_0) + \lambda R(\theta, \theta_0) $$

The idea is the seperate the two data groups not by $\theta^\top x^{(i)} - \theta_0 = 0$, but by $\theta^\top x^{(i)} - \theta_0 \geq 1$ and $\theta^\top x^{(i)} - \theta_0 \leq 1$. Hence, keep in mind that $\gamma_{\mathrm{ref}}$ is **not** like regular $\gamma$. We measure this by seeing the distance bewteen the two hyperplane as 
$$ \gamma_{\mathrm{ref}} = \frac 1 2 \frac 2 {\|\theta\|} $$

For loss function, we have several choice of the loss function $L$, one of them not in the clip is *hinge loss*:
$$ L_{\mathrm h}\left(\frac\gamma{\gamma_{\mathrm{ref}}}\right) = \begin{cases}1 - \frac\gamma{\gamma_{\mathrm{ref}}} & \text{if } \gamma < \gamma_{\mathrm{ref}} \\ 0 & \text{else} \end{cases} $$
With the loss in hand, we can write our objective as 
$$ J(\theta, \theta_0) = \frac 1 n \sum L_{\mathrm h}\left(\frac{y^{(i)}(\theta^\top x^{(i)} + \theta_0)}{\|\theta\|\gamma_{\mathrm{ref}}}\right) + \lambda \frac {1}{\gamma_{\mathrm{ref}}^2} $$
Because not only we want to minimize the classification error, we also want the margin to be as large as possible.

However, it is hard to directly apply optimization algorithms, such as gradient descent, with $\gamma_{\mathrm{ref}}$ in the formula. Fortunately, what defines the margin, geometrically, turns out to 
$$ \gamma_{\mathrm{ref}} = \frac 1 {\|\theta\|} $$

Hence we can obtain
$$ J(\theta, \theta_0) = \frac 1 n \sum L_{\mathrm h}(y^{(i)}(\theta^\top x^{(i)} + \theta_0)) + \lambda \|\theta\|^2 $$

Keep in mind that if $\lambda$ is zero, the $\theta$ will grow as much as possible as that *minimizes* our margin. Do note that first term is for *fitness*, and second term is for *significance*.

In [24]:
data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

def margin(data, label, th, th0):
    return label * (np.dot(data, th) + th0) / np.linalg.norm(th)

def hinge_loss(data, label, th, th0, g_ref):
    return np.float64(max(0, 1 - margin(data, label, th, th0) / g_ref))

loss = [hinge_loss(data[:,i], labels[:,i], th, th0, 1 / math.sqrt(2)) for i in range(len(data[0]))]
print(loss)

[0.7999999999999998, 0.0, 3.0]


### Part 6: SVM w/ Gradient Descent Implementation

In [119]:
# gradient descent
def gd(f, df, x0, step_size_fn, max_iter):
    xs = [x0]
    fs = [f(x0)]
    x = x0.copy()
    for i in range(1, max_iter + 1):
        x = x - step_size_fn(i) * df(x) 
        xs.append(x.copy())
        fs.append(f(x))
    return x, fs, xs

In [48]:
# numerical gradient approximation
def num_grad(f, delta=0.001):
    def df(x):
        d = len(x)
        def dx(i):
            zero = np.array([[0] * d]).T
            zero[i][0] = 1
            return zero
        return np.array([[(f(x + dx(i) * delta) - f(x - dx(i) * delta)) / (2 * delta) for i in range(d)]]).T
    return df


array([[ 6.999999, -2.      ]])

In [ ]:
# gradient descent w/ numerical gradient
def minimize(f, x0, step_size_fn, max_iter):
    df = num_grad(f)
    xs = [x0]
    fs = [f(x0)]
    x = x0.copy()
    for i in range(1, max_iter + 1):
        x = x - step_size_fn(i) * df(x) 
        xs.append(x.copy())
        fs.append(f(x))
    return x, fs, xs

In [110]:
# hinge loss of computed ratio (v)
def hinge(v):
    return np.where(v < 1, 1 - v, 0)

# hinge loss for seperator with a point
def hinge_loss(x, y, th, th0):
    return np.average(hinge(y * (np.dot(th.T, x) + th0)))

def svm_obj(x, y, th, th0, lam):
    return hinge_loss(x, y, th, th0) + lam * float(np.dot(th.T, th))



array([[0.15668397]])

In [131]:
# Returns the gradient of hinge(v) with respect to v.
def d_hinge(v):
    return np.where(v < 1, -1, 0)

# Returns the gradient of hinge_loss(x, y, th, th0) with respect to th
def d_hinge_loss_th(x, y, th, th0):
    return d_hinge(y * (np.dot(th.T, x) + th0)) * y * x

# Returns the gradient of hinge_loss(x, y, th, th0) with respect to th0
def d_hinge_loss_th0(x, y, th, th0):
    return d_hinge(y * (np.dot(th.T, x) + th0)) * y

# Returns the gradient of svm_obj(x, y, th, th0) with respect to th
def d_svm_obj_th(x, y, th, th0, lam):
    return np.array([np.average(d_hinge_loss_th(x, y, th, th0), axis=1)]).T + 2 * lam * th

# Returns the gradient of svm_obj(x, y, th, th0) with respect to th0
def d_svm_obj_th0(x, y, th, th0, lam):
    return np.array([np.average(d_hinge_loss_th0(x, y, th, th0), axis=1)])

# Returns the full gradient as a single vector (which includes both th, th0)
def svm_obj_grad(X, y, th, th0, lam):
    return np.vstack((d_svm_obj_th(X, y, th, th0, lam), d_svm_obj_th0(X, y, th, th0, lam)))


In [133]:
def batch_svm_min(data, labels, lam):
    def svm_min_step_size_fn(i):
       return 2/(i+1)**0.5
    def f(x):
        return svm_obj(data, labels, x[:-1], x[-1:], lam)
    def df(x):
        return svm_obj_grad(data, labels, x[:-1], x[-1:], lam)
    return gd(f, df, np.array([[0] * (len(data) + 1)]).T, svm_min_step_size_fn, 10)


In [134]:
def batch_svm_num_min(data, labels, lam):
    def svm_min_step_size_fn(i):
       return 2/(i+1)**0.5
    def f(x):
        return svm_obj(data, labels, x[:-1], x[-1:], lam)
    return num_grad(f, np.array([[0] * (len(data) + 1)]).T, svm_min_step_size_fn, 10)
